In [878]:
import pandas as pd

df = pd.read_csv(r'https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv', delimiter=';')
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [879]:
df[['y']].groupby('y').value_counts() #son muy pocas las personas en este dataframe que son target especifico de ofrecimiento de credito

y
no     36548
yes     4640
Name: count, dtype: int64

In [880]:
df.shape

(41188, 21)

In [881]:
df.rename(columns={'cons.price.idx': 'cons_price_idx'}, inplace=True)

In [882]:
df.rename(columns={'emp.var.rate': 'emp_var_rate'}, inplace=True)

In [883]:
df.rename(columns={'cons.conf.idx': 'cons_conf_idx'}, inplace=True)

In [884]:
df.rename(columns={'default': 'current_credit'}, inplace=True)

In [885]:
df.rename(columns={'nr.employed': 'nr_employed'}, inplace=True)

In [886]:
df['education'] = df['education'].str.replace('.', '_')

In [887]:
df['job'] = df['job'].str.replace('-', '_')

In [888]:
df['job'] = df['job'].str.replace('.', '_')

In [889]:
df.info()#hay varios valores que pueden ser cambiados a binario

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   current_credit  41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp_var_rate    41188 non-null  float64
 16  cons_price_idx  41188 non-null  float64
 17  cons_conf_idx   41188 non-null 

In [890]:
df.nunique()

age                 78
job                 12
marital              4
education            8
current_credit       3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp_var_rate        10
cons_price_idx      26
cons_conf_idx       26
euribor3m          316
nr_employed         11
y                    2
dtype: int64

In [891]:
df.drop(['month', 'duration', 'campaign', 'previous', 'poutcome', 'pdays', 'contact', 'day_of_week'],axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df #eliminando informacion irrelevante para que el modelo pueda predecir si la persona contrataria o no el deposito

,age,job,marital,education,current_credit,housing,loan,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic_4y,no,no,no,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high_school,unknown,no,no,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high_school,no,yes,no,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin_,married,basic_6y,no,no,no,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high_school,no,no,yes,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue_collar,married,professional_course,no,no,no,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university_degree,no,yes,no,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional_course,no,no,no,-1.1,94.767,-50.8,1.028,4963.6,yes


In [892]:
df.describe().T #tengo una edad minima que no aplica para obtener un producto financiero

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
emp_var_rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons_price_idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons_conf_idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr_employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


In [893]:
df.isnull().mean()*100 #mi dataset no tiene datos Nan

age               0.0
job               0.0
marital           0.0
education         0.0
current_credit    0.0
housing           0.0
loan              0.0
emp_var_rate      0.0
cons_price_idx    0.0
cons_conf_idx     0.0
euribor3m         0.0
nr_employed       0.0
y                 0.0
dtype: float64

In [894]:
df.isna().sum()

age               0
job               0
marital           0
education         0
current_credit    0
housing           0
loan              0
emp_var_rate      0
cons_price_idx    0
cons_conf_idx     0
euribor3m         0
nr_employed       0
y                 0
dtype: int64

In [895]:
#comienzo a filtrar primero por grupos de edad. Esto me indica que la edad minima es 17 y la maxima 98, el describe me indica que la medio por grupos percentiles de solo el 25% ya es de 32, y la del percentil 75 es de 47. 
#Con base en este informacion puedo delimitar el grupo de individuos cuya edad minima y maxima esten 5 unidades por encima y debajo de las medias de los cuartiles, es decir 25 y 62
age_population = df[['age']].groupby('age').value_counts()
age_population

age
17      5
18     28
19     42
20     65
21    102
     ... 
91      2
92      4
94      1
95      1
98      2
Name: count, Length: 78, dtype: int64

In [896]:
filtro_edad = df.query('age >= 18 and age <=98')
 #ahora tengo un rango de edad mas adecuado para contratar un producto financiero.

AQUI COMIENZO A VISUALIZAR LOS TIPOS DE CLASES QUE HAY EN CADA VARIABLE, A ELIMINAR VALORES UNKNOWN, Y A REEMPLAZARLOS POR VALORES BINARIOS.

In [897]:
filtro_edad[['job']].groupby('job').value_counts()

job
admin_           10422
blue_collar       9254
entrepreneur      1456
housemaid         1060
management        2924
retired           1720
self_employed     1421
services          3969
student            870
technician        6743
unemployed        1014
unknown            330
Name: count, dtype: int64

In [898]:
eliminar_trabajos = ['unknown'] 
gente_con_trabajo = filtro_edad[-filtro_edad['job'].isin(eliminar_trabajos)]
 #elimino unknown como datos Nan

In [899]:
gente_con_trabajo[['job']].groupby('job').value_counts()

job
admin_           10422
blue_collar       9254
entrepreneur      1456
housemaid         1060
management        2924
retired           1720
self_employed     1421
services          3969
student            870
technician        6743
unemployed        1014
Name: count, dtype: int64

In [900]:
gente_con_trabajo['job'] = gente_con_trabajo['job'].replace({'admin_': 0, 'blue_collar': 1, 'entrepreneur': 2, 'housemaid': 3, 'management': 4, 'retired': 5, 'self_employed': 6, 'services': 7, 'student': 8, 'technician': 9, 'unemployed': 10,})

/tmp/ipykernel_96834/874756226.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gente_con_trabajo['job'] = gente_con_trabajo['job'].replace({'admin_': 0, 'blue_collar': 1, 'entrepreneur': 2, 'housemaid': 3, 'management': 4, 'retired': 5, 'self_employed': 6, 'services': 7, 'student': 8, 'technician': 9, 'unemployed': 10,})
/tmp/ipykernel_96834/874756226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gente_con_trabajo['job'] = gente_con_trabajo['job'].replace({'admin_': 0, 'blue_collar': 1, 'entrepreneur': 2, 'ho

In [901]:
gente_con_trabajo[['marital']].groupby('marital').value_counts()#vuelvo a revisar para ver si los trabajos se eliminaron correctamente

marital
divorced     4599
married     24694
single      11489
unknown        71
Name: count, dtype: int64

In [902]:
eliminar_unknown = ['unknown']
marital = gente_con_trabajo[-gente_con_trabajo['marital'].isin(eliminar_unknown)]

In [903]:
marital['marital'] = marital['marital'].replace({'divorced': 0, 'married': 1, 'single':2})

/tmp/ipykernel_96834/1682185979.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  marital['marital'] = marital['marital'].replace({'divorced': 0, 'married': 1, 'single':2})
/tmp/ipykernel_96834/1682185979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marital['marital'] = marital['marital'].replace({'divorced': 0, 'married': 1, 'single':2})


In [904]:
unknown_default_cantidad = marital[['current_credit']].groupby('current_credit').value_counts()
unknown_default_cantidad
#para continuar necesito convertir todos los valores de los tipos de credito a binario 0 para si 1 para no, por tanto los datos unknown me estorban. Comienzo por la variable del credito default

current_credit
no         32343
unknown     8436
yes            3
Name: count, dtype: int64

In [905]:
eliminar_unknown = ['unknown']
gente_con_default = marital[-marital['current_credit'].isin(eliminar_unknown)]

In [906]:
gente_con_default['current_credit'] = gente_con_default['current_credit'].replace({'no': 0, 'yes': 1})

/tmp/ipykernel_96834/2728864360.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gente_con_default['current_credit'] = gente_con_default['current_credit'].replace({'no': 0, 'yes': 1})
/tmp/ipykernel_96834/2728864360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gente_con_default['current_credit'] = gente_con_default['current_credit'].replace({'no': 0, 'yes': 1})


In [907]:
#Ahora reviso la variable housing
unknown_housing_cantidad = gente_con_default[['housing']].groupby('housing').value_counts()
unknown_housing_cantidad #sigo teniendo datos inservibles por este lado, debo eliminarlos

housing
no         14492
unknown      760
yes        17094
Name: count, dtype: int64

In [908]:
eliminar_unknown = ['unknown']
gente_con_housing = gente_con_default[-gente_con_default['housing'].isin(eliminar_unknown)]

In [909]:
gente_con_housing['housing'] = gente_con_housing['housing'].replace({'no': 0, 'yes': 1})

/tmp/ipykernel_96834/765273244.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gente_con_housing['housing'] = gente_con_housing['housing'].replace({'no': 0, 'yes': 1})
/tmp/ipykernel_96834/765273244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gente_con_housing['housing'] = gente_con_housing['housing'].replace({'no': 0, 'yes': 1})


In [910]:
#ahora reviso los de loan
unknown_housing_loan = gente_con_housing[['loan']].groupby('loan').value_counts() #aqui ya no tengo unknown.
unknown_housing_loan

loan
no     26659
yes     4927
Name: count, dtype: int64

In [911]:
gente_con_housing['loan'] = gente_con_housing['loan'].replace({'no': 0, 'yes': 1})

/tmp/ipykernel_96834/2130042660.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gente_con_housing['loan'] = gente_con_housing['loan'].replace({'no': 0, 'yes': 1})
/tmp/ipykernel_96834/2130042660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gente_con_housing['loan'] = gente_con_housing['loan'].replace({'no': 0, 'yes': 1})


In [912]:
unknown_education = gente_con_housing[['education']].groupby('education').value_counts()
unknown_education

education
basic_4y                2380
basic_6y                1389
basic_9y                4274
high_school             7699
illiterate                11
professional_course     4321
university_degree      10412
unknown                 1100
Name: count, dtype: int64

In [913]:
eliminar_unknown_education = ['unknown']
gente_con_education = gente_con_housing[-gente_con_housing['education'].isin(eliminar_unknown_education)]
#este tipo de educacion no es adecuado para obtener un loan del banco.

In [914]:
gente_con_education['education'] = gente_con_education['education'].replace({'basic_4y': 0, 'basic_6y': 1, 'basic_9y': 2, 'high_school': 3, 'illiterate': 4, 'professional_course': 5, 'university_degree': 6})

/tmp/ipykernel_96834/3933186467.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gente_con_education['education'] = gente_con_education['education'].replace({'basic_4y': 0, 'basic_6y': 1, 'basic_9y': 2, 'high_school': 3, 'illiterate': 4, 'professional_course': 5, 'university_degree': 6})
/tmp/ipykernel_96834/3933186467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gente_con_education['education'] = gente_con_education['education'].replace({'basic_4y': 0, 'basic_6y': 1, 'basic_9y': 2, 'high_school': 3, 'illiter

In [915]:
gente_con_education[['y']].groupby('y').value_counts() #no hay unknown en esta variable y todavia conservamos buena parte de los target originales del dataframe

y
no     26627
yes     3859
Name: count, dtype: int64

In [916]:
gente_con_education['y'] = gente_con_education['y'].replace({'no': 0, 'yes': 1})
gente_con_education

/tmp/ipykernel_96834/2166326624.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gente_con_education['y'] = gente_con_education['y'].replace({'no': 0, 'yes': 1})
/tmp/ipykernel_96834/2166326624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gente_con_education['y'] = gente_con_education['y'].replace({'no': 0, 'yes': 1})


,age,job,marital,education,current_credit,housing,loan,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,3,1,0,0,0,0,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,1,0,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,1,1.1,93.994,-36.4,4.857,5191.0,0
6,59,0,1,5,0,0,0,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,1,5,0,1,0,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,1,1,5,0,0,0,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,5,1,6,0,1,0,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,9,1,5,0,0,0,-1.1,94.767,-50.8,1.028,4963.6,1


AQUI TERMINE DE FILTRAR Y REEMPLAZAR DATOS Y COMIENZO A ESTUDIARLOS PARA ENTRENAR EL MODELO

In [917]:
filtered_df = gente_con_education
sesgo = filtered_df.apply(lambda x: x.skew())
sesgo #hay un sesgo extremadamente negativo en default, lo cual no es normal, ya que el sesgo comun es de -2 a 2, debo revisar esa columna para establecer el porque de estos datos

age                 0.980833
job                 0.408341
marital            -0.162514
education          -0.363367
current_credit    100.796825
housing            -0.168019
loan                1.891989
emp_var_rate       -0.549059
cons_price_idx     -0.118469
cons_conf_idx       0.373983
euribor3m          -0.525227
nr_employed        -0.893966
y                   2.246196
dtype: float64

In [918]:
filtered_df[['current_credit']].groupby('current_credit').value_counts() #esto me muestra la razon del sesgo negativo extremadamente alto y es por que hay una distribucion de datos desequilibrada, pero esto no significa un problema para continuar con el eda.
#este valor en fault muestra que entre el grupo de datos, solo dos personas tienes producto tipo default.

current_credit
0    30483
1        3
Name: count, dtype: int64

In [919]:
housing_0 = filtered_df.copy()
housing_0

,age,job,marital,education,current_credit,housing,loan,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,3,1,0,0,0,0,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,1,0,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,1,1.1,93.994,-36.4,4.857,5191.0,0
6,59,0,1,5,0,0,0,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,1,5,0,1,0,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,1,1,5,0,0,0,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,5,1,6,0,1,0,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,9,1,5,0,0,0,-1.1,94.767,-50.8,1.028,4963.6,1


In [920]:
#housing_0[['euribor3m']].groupby('euribor3m').value_counts() #de 4600 personas que eran originalmente target, aqui tengo aun casi 2142, casi una cuarta parte de eso. Estos filtros me sirven para buscar nuevos prospectos con caracteristicas similares a los target
#En este caso, buena parte de los target tienen credito hipotecario, tienen trabajos un tanto mas estables, se entiende que estas sean las razones de ser target.
#variable y solo tiene dos opciones,
if 'unknown' in housing_0['job'].values:
    print("La columna contiene valores 'unknown'.")
else:
    print("La columna no contiene valores 'unknown'.")

La columna no contiene valores 'unknown'.


In [921]:
housing_0[['job']].groupby('job').value_counts()

job
0     8737
1     5675
2     1089
3      690
4     2311
5     1216
6     1092
7     2857
8      608
9     5473
10     738
Name: count, dtype: int64

In [922]:
# Calcular el monto máximo del préstamo usando los valores de euribor3m como coeficientes
# Calcular el precio medio del euribor por cada tipo de trabajo
housing_0['employees_per_job'] = housing_0.groupby('job')['nr_employed'].transform('mean').round()
housing_0.drop(['nr_employed'], axis=1, inplace=True) #el numero de empleados por tipo de trabajo tiene una correlacion alta con otras variables
#housing_0['conf_emp_ratio'] = housing_0['cons_conf_idx'] / df['emp_var_rate']
education_emp_var_mean = housing_0.groupby('education')['emp_var_rate'].mean()
housing_0['education_emp_var_mean'] = housing_0['education'].map(education_emp_var_mean)
#education_confidence_mean = housing_0.groupby('education')['cons_conf_idx'].mean()
#housing_0['education_confidence_mean'] = housing_0['education'].map(education_confidence_mean)
job_age_mean = housing_0.groupby('job')['age'].mean()
housing_0['job_age_mean'] = housing_0['job'].map(job_age_mean)
#housing_0['credit_conf_difference'] = housing_0['current_credit'] - housing_0['cons_conf_idx']
#housing_0['credit_conf_ratio'] = housing_0['current_credit'] / housing_0['cons_conf_idx']
# Suponiendo que tengas un DataFrame df y deseas calcular la proporción entre cons_conf_idx y emp_var_rate
#housing_0['conf_emp_ratio'] = housing_0['cons_conf_idx'] / housing_0['emp_var_rate']
credit_acceptance_prop = housing_0.groupby('current_credit')['y'].mean()
credit_acceptance_dict = credit_acceptance_prop.to_dict()
housing_0['credit_acceptance_prop'] = housing_0['current_credit'].map(credit_acceptance_dict)
marital_conf_mean = housing_0.groupby('marital')['cons_conf_idx'].mean()
marital_conf_dict = marital_conf_mean.to_dict()
housing_0['marital_conf_mean'] = housing_0['marital'].map(marital_conf_dict)
housing_0

,age,job,marital,education,current_credit,housing,loan,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,y,employees_per_job,education_emp_var_mean,job_age_mean,credit_acceptance_prop,marital_conf_mean
0,56,3,1,0,0,0,0,1.1,93.994,-36.4,4.857,0,5170.0,-0.175546,44.675362,0.126595,-40.361988
2,37,7,1,3,0,1,0,1.1,93.994,-36.4,4.857,0,5166.0,-0.089570,36.760938,0.126595,-40.361988
3,40,0,1,1,0,0,0,1.1,93.994,-36.4,4.857,0,5159.0,0.030382,37.544352,0.126595,-40.361988
4,56,7,1,3,0,0,1,1.1,93.994,-36.4,4.857,0,5166.0,-0.089570,36.760938,0.126595,-40.361988
6,59,0,1,5,0,0,0,1.1,93.994,-36.4,4.857,0,5159.0,0.056353,37.544352,0.126595,-40.361988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,1,5,0,1,0,-1.1,94.767,-50.8,1.028,1,5104.0,0.056353,63.199836,0.126595,-40.361988
41184,46,1,1,5,0,0,0,-1.1,94.767,-50.8,1.028,0,5168.0,0.056353,38.164229,0.126595,-40.361988
41185,56,5,1,6,0,1,0,-1.1,94.767,-50.8,1.028,0,5104.0,-0.133356,63.199836,0.126595,-40.361988
41186,44,9,1,5,0,0,0,-1.1,94.767,-50.8,1.028,1,5172.0,0.056353,37.602777,0.126595,-40.361988


In [923]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcula la matriz de correlación
corr= housing_0.corr()

# Crea el mapa de correlación 
corr.style.background_gradient(cmap='coolwarm').format(precision=3)

,age,job,marital,education,current_credit,housing,loan,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,y,employees_per_job,education_emp_var_mean,job_age_mean,credit_acceptance_prop,marital_conf_mean
age,1.000,-0.005,-0.401,-0.130,0.003,0.005,-0.011,-0.051,-0.036,0.125,-0.037,0.049,-0.196,-0.059,0.530,-0.003,0.348
job,-0.005,1.000,0.019,0.104,0.015,0.006,-0.019,-0.003,-0.017,0.042,-0.004,0.021,-0.035,0.231,-0.010,-0.015,-0.037
marital,-0.401,0.019,1.000,0.113,-0.003,0.009,0.004,-0.071,-0.050,-0.038,-0.081,0.038,-0.042,-0.041,-0.188,0.003,-0.615
education,-0.130,0.104,0.113,1.000,0.002,0.020,0.009,-0.007,-0.064,0.084,0.000,0.041,0.043,-0.154,-0.097,-0.002,-0.146
current_credit,0.003,0.015,-0.003,0.002,1.000,-0.004,-0.004,0.006,-0.003,0.005,0.007,-0.004,0.001,0.010,-0.002,-1.000,0.008
housing,0.005,0.006,0.009,0.020,-0.004,1.000,0.047,-0.059,-0.077,-0.036,-0.058,0.010,-0.001,0.009,-0.000,0.004,-0.009
loan,-0.011,-0.019,0.004,0.009,-0.004,0.047,1.000,-0.000,-0.002,-0.019,-0.003,-0.005,-0.001,-0.004,-0.014,0.004,0.000
emp_var_rate,-0.051,-0.003,-0.071,-0.007,0.006,-0.059,-0.000,1.000,0.766,0.158,0.969,-0.305,0.197,0.048,-0.074,-0.006,0.065
cons_price_idx,-0.036,-0.017,-0.050,-0.064,-0.003,-0.077,-0.002,0.766,1.000,0.028,0.667,-0.129,0.094,0.042,-0.039,0.003,0.036
cons_conf_idx,0.125,0.042,-0.038,0.084,0.005,-0.036,-0.019,0.158,0.028,1.000,0.244,0.062,-0.078,-0.049,0.085,-0.005,0.061


In [924]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.Series([variance_inflation_factor(corr.values, i) for i in range(corr.shape[1])], index=housing_0.columns)
vif #multicolinealidad alta

/home/vscode/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


age                       1.162103e+02
job                       1.447296e+00
marital                   6.543965e+00
education                 1.623141e+00
current_credit                     inf
housing                   1.079467e+00
loan                      1.016314e+00
emp_var_rate              4.735795e+03
cons_price_idx            1.427759e+02
cons_conf_idx             6.248981e+02
euribor3m                 1.253787e+04
y                         4.417331e+01
employees_per_job         1.168143e+01
education_emp_var_mean    1.640346e+00
job_age_mean              4.776645e+00
credit_acceptance_prop             inf
marital_conf_mean         5.970817e+00
dtype: float64

la vairable euribor3m es importante por cuanto ayudara al modelo a predecir el monto minimo de deposito //
la variable de la tasa de variacion de empleados por trabajo es importante debido a que es un indicador de si la economia esta creciendo o decreciendo para el prospecto, lo cual influye en su decision de aceptar hacer el deposito o no //

In [925]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score#

X = housing_0.drop(['y'], axis=1)  # Características (todas las columnas excepto la columna 'y')
y = housing_0['y']                  # Etiquetas (columna 'y')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo de regresión logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train_scaled, y_train)

# Evaluar el modelo
accuracy = logistic_model.score(X_test_scaled, y_test)
print("Accuracy:", accuracy*100)
#un 90% de probabilidad de que los prospecto seleccionados, bajo los terminos que se han propuesto, acepten el prestamo
#voy a revisar las metricas de evaluacion de clasificacion

Accuracy: 87.37290915054116


In [926]:
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de y_train:", y_train.shape)
print("Tamaño de X_test:", X_test.shape)
print("Tamaño de y_test:", y_test.shape)

Tamaño de X_train: (24388, 16)
Tamaño de y_train: (24388,)
Tamaño de X_test: (6098, 16)
Tamaño de y_test: (6098,)


In [927]:
from sklearn.metrics import confusion_matrix

# Predecir con el modelo
y_pred = logistic_model.predict(X_test_scaled)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

print("Matriz de Confusión:")
print(conf_matrix)

Matriz de Confusión:
[[5245   77]
 [ 693   83]]


True negatives: 5249 //
False positives: 73 //
False negatives: 690 //
True positives: 86 //

In [928]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcular la precisión
precision = precision_score(y_test, y_pred)

# Calcular el recall
recall = recall_score(y_test, y_pred)

# Calcular el F1-score
f1 = f1_score(y_test, y_pred)

print("Precisión:", precision*100) #las muestras que fueron correctas entre todas las predicciones positivas
print("Recall:", recall*100) #sensibilidad del modelo y las muestras que fueron clasificadas correctamente entre todas las muestras positivas reales
print("F1-score:", f1*100) #media entre las dos anteriores

Precisión: 51.87500000000001
Recall: 10.695876288659793
F1-score: 17.735042735042736


In [929]:
probs = logistic_model.predict_proba(X_test_scaled)

# Crear un nuevo DataFrame con las probabilidades convertidas a porcentaje y redondeadas
probs_df = pd.DataFrame(probs * 100, columns=['prob_class_0', 'prob_class_1'], index=X_test.index).round(2)

# Concatenar el DataFrame de probabilidades con el DataFrame original
X_test_with_probs = pd.concat([X_test, probs_df], axis=1).round()
X_test_with_probs


,age,job,marital,education,current_credit,housing,loan,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,employees_per_job,education_emp_var_mean,job_age_mean,credit_acceptance_prop,marital_conf_mean,prob_class_0,prob_class_1
8688,27,7,2,3,0,1,0,1.0,94.0,-42.0,5.0,5166.0,-0.0,37.0,0.0,-41.0,94.0,6.0
18409,27,1,2,0,0,0,0,1.0,94.0,-43.0,5.0,5168.0,-0.0,38.0,0.0,-41.0,97.0,3.0
18374,37,2,0,3,0,1,0,1.0,94.0,-43.0,5.0,5172.0,-0.0,41.0,0.0,-41.0,97.0,3.0
39804,29,9,2,5,0,1,0,-2.0,94.0,-40.0,1.0,5172.0,0.0,38.0,0.0,-41.0,55.0,45.0
10746,28,7,2,3,0,0,0,1.0,94.0,-42.0,5.0,5166.0,-0.0,37.0,0.0,-41.0,94.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,30,0,2,3,0,1,1,1.0,94.0,-36.0,5.0,5159.0,-0.0,38.0,0.0,-41.0,94.0,6.0
22951,32,9,0,3,0,1,1,1.0,93.0,-36.0,5.0,5172.0,-0.0,38.0,0.0,-41.0,97.0,3.0
27332,35,0,0,6,0,1,0,-0.0,93.0,-42.0,4.0,5159.0,-0.0,38.0,0.0,-41.0,94.0,6.0
20781,41,9,1,5,0,0,0,1.0,93.0,-36.0,5.0,5172.0,0.0,38.0,0.0,-40.0,97.0,3.0


In [930]:
dfx = X_test_with_probs[['prob_class_1']].groupby('prob_class_1').value_counts()
dfx

prob_class_1
0.0       1
2.0      32
3.0     805
4.0     900
5.0     725
       ... 
62.0      4
63.0      4
64.0      3
66.0      2
67.0      2
Name: count, Length: 66, dtype: int64

26.0 mediana
prob_class_1
0.0       1
2.0      32
3.0     805
4.0     900
5.0     725
       ... 
62.0      4
63.0      4
64.0      3
66.0      2
67.0      2

prob_class_1
0.0       1
2.0      28
3.0     835
4.0     878
5.0     718
       ... 
63.0      3
64.0      2
65.0      1
66.0      2
67.0      1
25.0 mediana

datos iniciales:
Precisión: 51.28205128205128
Recall: 8.403361344537815
F1-score: 14.440433212996389